# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

In [3]:
RECORDING_DIR = Path('H:/Recordings/')

## Data Loading

Firstly we need to import all the recordings into the notebook.


In [4]:
from dataset import FESDDataset
from utils import Frame, AugmentationParams
import json
import numpy as np

import cv2

h:\FESD\FESDModel\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

In [6]:
dataset = FESDDataset(RECORDING_DIR, 500)

Recordings Found: 25
Total Frames: 7500


In [7]:
dataset.randomize_augmentation_params = True
sample_frame = dataset[0]
sample_frame.show()

print("All Missing Joints")
print(sample_frame.errors[sample_frame.errors==1])
print("All Wrong Joints")
print(sample_frame.poses_2d[sample_frame.errors==2])

AugmentationParams(flip=False, crop=True, crop_random=True, crop_pad=40, gaussian=True, seed=39479)
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_0.yml
All Missing Joints
[1 1 1 1 1]
All Wrong Joints
[[ -1.42166138 167.15560913   0.2603097 ]
 [ -1.1519165  170.9956665    0.25113821]]


### Build Model

The model proposed by RD3D is based on resnet50 so we copy a pretrained resnet50 model

In [8]:
from model import RD3D
import copy

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

num_gpus = torch.cuda.device_count()
print(f"Num cuda GPUs: {num_gpus}")

Num cuda GPUs: 1


In [9]:
resnet = torchvision.models.resnet50(pretrained=True)

model = RD3D(32, copy.deepcopy(resnet))
print(model)

model = nn.DataParallel(model).cuda()

h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RD3D(
  (resnet): I3DResNet(
    (conv1): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): 

## Train Model

In the following we define the training function and train a network on the training data.

In [10]:
batchsize = 32
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize, shuffle=True, num_workers=0)

In [12]:
# training
def train_salient(train_loader, model, optimizer, criterion, scheduler, epoch, opt):
    # multi-scale training  
    size_rates = [0.75, 1, 1.25]

    model.train()
    loss_record = AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        # for rate in size_rates:
        #     optimizer.zero_grad()
        #     images, gts, depths = pack
        #     images = images.cuda()
        #     gts = gts.cuda()
        #     depths = depths.cuda()

        #     # multi-scale training samples
        #     trainsize = int(round(opt.trainsize * rate / 32) * 32)
        #     if rate != 1:
        #         images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
        #         images = images.unsqueeze(2)
        #         gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)

        #         depths = F.upsample(depths, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
        #         depths = depths.unsqueeze(2)
        #         images = torch.cat([images, depths], 2)

        #     if rate == 1:
        #         images = images.unsqueeze(2)
        #         depths = depths.unsqueeze(2)
        #         images = torch.cat([images, depths], 2)

        #     # forward
        #     pred_s = model(images)
        #     # TODO Calculate different loss based on the error label
        #     loss = criterion(pred_s, gts)

        #     loss.backward()
        #     clip_gradient(optimizer, opt.clip)
        #     optimizer.step()
        #     scheduler.step()
        #     if rate == 1:
                
        #         loss_record.update(loss.data, opt.batchsize)
      
        if i % 100 == 0 or i == len(train_loader):
            logger.info('Epoch [{:03d}/{:03d}], Step [{:04d}/{:04d}], Loss: {:.4f}'.
                        format(epoch, opt.epochs, i, len(train_loader),
                               loss_record.show()))